In [63]:
import numpy as np
import pandas as pd
from itertools import product
from tqdm import tqdm

In [64]:
item_categories = pd.read_csv('data/item_categories.csv')
items = pd.read_csv('data/items.csv')
sales_train = pd.read_csv('data/sales_train.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')
shops = pd.read_csv('data/shops.csv')
test = pd.read_csv('data/test.csv')

datasets = {
    'item_categories': item_categories.columns.values,
    'items': items.columns.values,
    'sales_train': sales_train.columns.values,
    'sample_submission': sample_submission.columns.values,
    'shops': shops.columns.values,
    'test': test.columns.values
}

# Si cada dataset tiene menos de 6 columnas, completar con NaN.
for dataset in datasets:
    while len(datasets[dataset]) < 6:
        datasets[dataset] = np.append(datasets[dataset], '-')

pd.DataFrame(datasets)

,item_categories,items,sales_train,sample_submission,shops,test
0,item_category_name,item_name,date,ID,shop_name,ID
1,item_category_id,item_id,date_block_num,item_cnt_month,shop_id,shop_id
2,-,item_category_id,shop_id,-,-,item_id
3,-,-,item_id,-,-,-
4,-,-,item_price,-,-,-
5,-,-,item_cnt_day,-,-,-


In [65]:
max_date = pd.to_datetime(sales_train['date'], format='%d.%m.%Y').max()
# min_date = pd.to_datetime(sales_train['date'], format='%d.%m.%Y').min()
min_date = max_date - pd.Timedelta(days=730)
dates = pd.date_range(start=min_date, end=max_date, freq='D')

dfprod = pd.DataFrame({
    'date': pd.to_datetime(sales_train['date'], format='%d.%m.%Y'),
    'day': pd.to_datetime(sales_train['date'], format='%d.%m.%Y').dt.day,
    'month': pd.to_datetime(sales_train['date'], format='%d.%m.%Y').dt.month,
    'quarter': pd.to_datetime(sales_train['date'], format='%d.%m.%Y').dt.quarter,
    'year': pd.to_datetime(sales_train['date'], format='%d.%m.%Y').dt.year,
    'weekday': pd.to_datetime(sales_train['date'], format='%d.%m.%Y').dt.weekday,
    'is_month_start': pd.to_datetime(sales_train['date'], format='%d.%m.%Y').dt.is_month_start,
    'is_month_end': pd.to_datetime(sales_train['date'], format='%d.%m.%Y').dt.is_month_end,
    'acc_month_num': (pd.to_datetime(sales_train['date'], format='%d.%m.%Y').dt.year - min_date.year) * 12 + pd.to_datetime(sales_train['date'], format='%d.%m.%Y').dt.month - min_date.month,
    'shop_id': sales_train['shop_id'],
    'item_id': sales_train['item_id'],
    'category_id': pd.merge(sales_train, items, on='item_id', how='left')['item_category_id'],
    'price': sales_train['item_price'],
    'quantity': sales_train['item_cnt_day']
})

pbirusia = pd.DataFrame([
    [396372, 410803, 413837, 341693, 287406, 351676, 299481, 292302]
], columns=['2014T1', '2014T2', '2014T3', '2014T4', '2015T1', '2015T2', '2015T3', '2015T4'])
for quarter, id in enumerate(pbirusia):
    dfprod.loc[dfprod['quarter'] == quarter + 1, 'pbi'] = pbirusia[id].values[0]

opep = pd.read_csv('barrilOPEP.csv', header=0)
opep['date'] = pd.to_datetime(opep['date'], format='%Y-%m-%d')
dfprod = pd.merge(dfprod, opep, on='date', how='left')
dfprod['OPEP_oil_price'] = dfprod['OPEP_oil_price'].fillna(method='ffill')

urate = pd.read_csv('desempleo.csv', header=0)
dfprod = pd.merge(dfprod, urate, on=['month', 'year'], how='left')

IPCrate = pd.read_csv('inflacion.csv', header=0)
dfprod = pd.merge(dfprod, IPCrate, on=['month', 'year'], how='left')

ventas = dfprod.groupby(['shop_id', 'item_id']).size().reset_index(name='size')
ventasfilt = ventas.loc[(ventas['size'] > 20)]
dfprod = pd.merge(dfprod, ventasfilt, on=['item_id', 'shop_id']).drop(['size'], axis=1)
dfprod = dfprod.loc[(dfprod['date'] >= min_date)]

/var/folders/x4/xws1vb114617s04w397tyhj80000gn/T/ipykernel_27008/3544733977.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dfprod['OPEP_oil_price'] = dfprod['OPEP_oil_price'].fillna(method='ffill')


In [66]:
dfprod.sample(5)

,date,day,month,quarter,year,weekday,is_month_start,is_month_end,acc_month_num,shop_id,item_id,category_id,price,quantity,pbi,OPEP_oil_price,unemployment_rate,IPC_rate
678807,2014-10-30,30,10,4,2014,3,False,False,12,56,15063,30,449.0,1.0,341693.0,82.79,5.1,8.3
590160,2013-10-31,31,10,4,2013,3,False,True,0,25,11498,37,499.0,1.0,341693.0,106.75,5.5,6.3
220057,2014-01-06,6,1,1,2014,0,False,False,3,31,7977,33,1099.0,1.0,396372.0,104.33,5.6,6.1
52435,2015-01-18,18,1,1,2015,6,False,False,15,19,13511,19,1499.0,1.0,396372.0,43.69,5.5,15.0
961490,2014-08-19,19,8,3,2014,1,False,False,10,29,5243,23,1110.6,1.0,413837.0,98.93,4.8,7.6


In [67]:
# Obtén todos los productos únicos
todos_productos = dfprod[['shop_id', 'item_id']].drop_duplicates().assign(key=range(1, len(dfprod[['shop_id', 'item_id']].drop_duplicates()) + 1))
todas_fechas = pd.DataFrame({'date': dates})

# Divide las fechas en lotes
batch_size = 100  # ajusta según tu capacidad de memoria
fechas_lotes = [todas_fechas[i:i + batch_size] for i in range(0, todas_fechas.shape[0], batch_size)]

# Crea un DataFrame con todas las combinaciones de fechas y item_id
for i, fechas_lote in tqdm(enumerate(fechas_lotes)):
    # Realiza un left join con las ventas reales
    all_combinations = pd.DataFrame(list(product(fechas_lote['date'], todos_productos['key'])), columns=['date', 'key'])
    all_combinations = pd.merge(all_combinations, todos_productos, on='key', how='left').drop('key', axis=1)
    merged_df = pd.merge(all_combinations, dfprod, on=['date', 'shop_id', 'item_id'], how='left')

    # Rellenar NaN con 0 en la columna 'quantity'
    merged_df['quantity'].fillna(0, inplace=True)

    if i == 0:
        merged_df.to_csv('data/dataset.csv', index=False)
    else:
        merged_df.to_csv('data/dataset.csv', mode='a', header=False, index=False)



8it [01:24, 10.58s/it]
